In [5]:
import imgaug.augmenters as iaa
import numpy as np
import cv2
import os

In [1]:
import tensorflow as tf
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Paths to the images and masks
images_folder_path = 'input'
masks_folder_path = 'output'

In [6]:
# Load the images and masks
def load_images(image_path, mask_path):
    images, masks, fnames = [], [], []
    for image_name in sorted(os.listdir(image_path)):
        image = cv2.imread(os.path.join(image_path, image_name))
        # Resize the image to 256x256
        image = cv2.resize(image, (256, 256))
        # Load the mask
        mask_name = image_name.replace(".png", ".tif")
        mask = cv2.imread(os.path.join(mask_path, mask_name))
        # Resize the mask to 256x256
        mask = cv2.resize(mask, (256, 256))
        images.append(image)
        masks.append(mask)
        fnames.append(image_name)
    return images, masks, fnames

In [7]:
def augmentation_fn(image, mask, training=True):
    img_aug = image.copy()
    mask_aug = mask.copy()

    if training:
        # randomly change the brightness
        img_aug = tf.image.random_brightness(img_aug, 0.1)
        # randomly change the hue
        img_aug = tf.image.random_hue(img_aug, 0.1)

        # randomly flip the image horizontally
        if np.random.rand() > 0.5:
            img_aug = tf.image.flip_left_right(img_aug)
            mask_aug = tf.image.flip_left_right(mask_aug)

        # randomly flip the image vertically
        if np.random.rand() > 0.5:
            img_aug = tf.image.flip_up_down(img_aug)
            mask_aug = tf.image.flip_up_down(mask_aug)


    # Normalization factor for scaling images
    normalization_factor = 122.5

    # Resize and normalize the image to the range [-1, 1]
    image_resized_normalized = (tf.image.resize(image, (128, 128)) / normalization_factor) - 1.0

    # Resize the mask and apply a threshold to create a binary mask
    mask_resized = tf.image.resize(mask, (128, 128))
    mask_resized_binary = tf.cast(mask_resized > normalization_factor, tf.float32)

    # Ensure mask has the correct shape by adding a channel dimension
    mask_final = mask_resized_binary[:, :, :1]

    return image_resized_normalized, mask_final